# #002_DL_2

In [25]:
import tensorflow as tf

In [26]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.imdb.load_data()

In [27]:
dir(tf.keras.datasets.imdb)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'get_word_index',
 'load_data']

In [28]:
for i,j in tf.keras.datasets.imdb.get_word_index().items():
    if j==503:
        print(i)

child


In [29]:
X_train.shape

(25000,)

In [30]:
len(X_train[0])

218

In [13]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [8]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [37]:
y_train.shape

(25000,)

각 영화마다 평가의 수가 다르기 때문에 다 같은 사이즈로 변경시켜줘야한다.
- 원핫인코딩 : 백터로 변경하지만 데이터가 너무 많기 때문에 옳지않다.
- 임베딩 : 가장 큰 배열 사이즈로 통일시키고, 패딩(여백)을 추가한다.

In [36]:
#모델 구성(방법 2)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(25000, 16, input_shape=(None,)))
model.add(tf.keras.layers.GlobalAveragePooling1D()) # 이미지 압축하는 기술 : 풀링
model.add(tf.keras.layers.Dense(16, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)) #sigmoid는 0과1 분류이다.(only binary)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 16)          400000    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 400,289
Trainable params: 400,289
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(optimizer=tf.train.AdamOptimizer(), #객체로 접근하면 init설정도 바꿀수있고, 상속으로 커스텀도 가능하다.
              loss='binary_crossentropy',
              metrics=['acc']) #단축표현 제공

---

In [47]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [53]:
data_path = tf.keras.utils.get_file('auto-mpg.data',
                               'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data')
data_path

'/Users/pd/.keras/datasets/auto-mpg.data'

In [57]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(data_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

data = raw_dataset.copy()
# null값이 있을때, 사이킷은 오류를 내고 케라스는 학습이 안된다.

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
MPG             398 non-null float64
Cylinders       398 non-null int64
Displacement    398 non-null float64
Horsepower      392 non-null float64
Weight          398 non-null float64
Acceleration    398 non-null float64
Model Year      398 non-null int64
Origin          398 non-null int64
dtypes: float64(5), int64(3)
memory usage: 25.0 KB


In [68]:
data.Origin = data.Origin.map({'usa':0,'japan':1,'europe':2})

In [63]:
tf.keras.utils.to_categorical(data.Origin.values)

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)